## Imports

In [1]:
%load_ext autotime

import numpy as np
from sklearn.ensemble import AdaBoostClassifier

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Input

In [2]:
data_matrix = np.matrix([[1., 2.1],
                     [2., 1.1],
                     [1.3, 1.],
                     [1., 1.],
                     [2., 1.]])
class_labels = np.matrix([1.0, 1.0, -1.0, -1.0, 1.0])

time: 2.02 ms


In [3]:
rows, cols = np.shape(data_matrix)

time: 141 ms


In [4]:
sample_weight = np.ones((rows, 1)) / rows
sample_weight

array([[0.2],
       [0.2],
       [0.2],
       [0.2],
       [0.2]])

time: 159 ms


## Lets work on the first feature (first column)

***Set the minError to infinity <br>
For every feature in the dataset: <br>
&nbsp;&nbsp;For every step: <br>
&nbsp;&nbsp;&nbsp;&nbsp;For each inequality: <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Build a decision stump and test it with the weighted dataset <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If the error is less than minError: set this stump as the best stump <br>
Return the best stump***

In [5]:
num_steps = 10
print(f"First column is : \n{data_matrix[:, 0]}")
col_min = data_matrix[:, 0].min()
col_max = data_matrix[:, 0].max()
print(f"min being : {col_min}")
print(f"max being : {col_max}")
step_size = (col_max - col_min) / num_steps
print(f"Step size : {step_size}")

First column is : 
[[1. ]
 [2. ]
 [1.3]
 [1. ]
 [2. ]]
min being : 1.0
max being : 2.0
Step size : 0.1
time: 2.39 ms


In [6]:
thresh_val = (col_min + float(0) * step_size)
thresh_val

1.0

time: 133 ms


Create a threshold using the step size and the column that you are working with. <br>
Create prediction just by seeing if a value is less than or greater than this threshold. We do this because, we are trying to create stumps (decision trees with one node and two leaves). <br>

In [7]:
prediction = np.ones((rows, 1))
prediction[data_matrix[:, 0] <= thresh_val] = -1.0
prediction

array([[-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.]])

time: 3.97 ms


Check these prdictions with the actual class labels.

In [11]:
error = np.matrix(np.ones((rows, 1)))
error[prediction == class_labels.T] = 0
error

matrix([[1.],
        [0.],
        [1.],
        [0.],
        [0.]])

time: 7.71 ms


get weighted error by using error matrix and initial sample weight

In [12]:
weighted_error = sample_weight.T * error
weighted_error

matrix([[0.4]])

time: 4.17 ms


## Putting it all together

In [55]:
def stump_classify(data_mat, column_ind, thresh, thresh_inequality):
    rows, cols = np.shape(data_mat)
    prediction = np.ones((rows, 1))
    if thresh_inequality == "lt":
        prediction[data_mat[:, column_ind] <= thresh] = -1
    else:
        prediction[data_mat[:, column_ind] > thresh] = -1
    return prediction

time: 1.63 ms


In [14]:
def build_stump(data_arr, label_matrix, sample_weight, num_steps=10):
    data_matrix = np.mat(data_arr)
    label_matrix = np.mat(label_matrix).T
    rows, cols = np.shape(data_matrix)
    best_class_est = np.mat(np.zeros((rows, 1)))
    best_stump, min_error = dict(), float("Inf")
    
    for col_index in range(cols):#loop over all features
        range_min = data_matrix[:,col_index].min()
        range_max = data_matrix[:,col_index].max()
        step_size = (range_max - range_min) / num_steps
        
        for epoch in range(num_steps + 1):# loops to get different threshold values
            for inequality in ['lt', 'gt']: # checks both inequalities
                thresh = (range_min + float(epoch) * step_size)
                prediction = stump_classify(data_matrix, col_index, thresh, inequality)
                
                error = np.mat(np.ones((rows, 1)))
                error[prediction == label_matrix] = 0
                weighted_error = sample_weight.T * error
                
                print(f"min error: {min_error}")
                if weighted_error.item(0) < min_error:
                    min_error = weighted_error
                    best_class_est = prediction.copy()
                    best_stump['dim'] = col_index
                    best_stump['thresh'] = thresh
                    best_stump['ineq'] = inequality
                    
    return best_stump, min_error, best_class_est

time: 5.92 ms


In [16]:
sample_weight = np.ones((rows, 1)) / rows
build_stump(data_matrix, class_labels, sample_weight, num_steps=10)

min error: inf
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]


({'dim': 0, 'thresh': 1.3, 'ineq': 'lt'},
 matrix([[0.2]]),
 array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

time: 30.6 ms


## Implementing AdaBoost to use multiple weak learners

In [21]:
def adaboost(data_arr, class_labels, epochs=40):
    weak_class_arr = list()
    rows, cols = np.shape(data_arr)
    sample_weight = np.mat(np.ones((rows, 1)) / rows)   
    agg_class_est = np.mat(np.zeros((rows, 1)))
    
    for epoch in range(epochs):
        best_stump, error, class_est = build_stump(data_arr, class_labels, sample_weight)
#         print("sample_weight:",sample_weight.T)
        
        alpha = float(0.5 * np.log((1.0 - error) / max(error, 1e-16)))
        best_stump['alpha'] = alpha  
        weak_class_arr.append(best_stump)
        print("class_est: ",class_est.T)
        
        expon = np.multiply(-1 * alpha * np.mat(class_labels).T, class_est)
        sample_weight = np.multiply(sample_weight, np.exp(expon))
        print(f"sample weight: {sample_weight}")
        sample_weight = sample_weight / sample_weight.sum()

        agg_class_est += alpha * class_est
        print("agg_class_est: ", agg_class_est.T)
        agg_errors = np.multiply(np.sign(agg_class_est) != np.mat(class_labels).T, np.ones((rows,1)))
        error_rate = agg_errors.sum() / rows
        print("total error: ", error_rate)
        if error_rate == 0.0: break
    return weak_class_arr

time: 5.94 ms


In [23]:
classifier_array = adaboost(data_matrix, class_labels, epochs=40)

min error: inf
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.4]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
min error: [[0.2]]
class_est:  [[-1.  1. -1. -1.  1.]]
sample weight: [[0.4]
 [0.1]
 [0.1]
 [0.1]
 [0.1]]
agg_class_est:  [[-0.69314718  0.69314718 -0.69314718 -0.69314718  0.69314718]]
t

In [24]:
classifier_array

[{'dim': 0, 'thresh': 1.3, 'ineq': 'lt', 'alpha': 0.6931471805599453},
 {'dim': 1, 'thresh': 1.0, 'ineq': 'lt', 'alpha': 0.9729550745276565},
 {'dim': 0, 'thresh': 2.0, 'ineq': 'gt', 'alpha': 0.8958797346140273}]

time: 4.87 ms


## Classification using AdaClassifier

In [63]:
def ada_classify(data_to_classify, classifier_array):
    data_mat = np.mat(data_to_classify)
    rows, cols = np.shape(data_mat)
    agg_class_est = np.zeros((rows, 1))
    
    for ind, classifier in enumerate(classifier_array):
        class_est = stump_classify(data_mat, classifier["dim"], classifier["thresh"], classifier["ineq"])
        print(agg_class_est)
        agg_class_est += classifier['alpha'] * class_est
    
    return np.sign(agg_class_est)

time: 1.42 ms


We will use the same stump classify function that we had developed nbefore to classify new data point. We pass it through all classifiers, and we return the sign of the aggregated weighted result (class estimation multiplied by classifier alpha).<br>
We can see that the error is becoming more and more negative/positive with each iteration (each pass through a week classifier)!

In [65]:
ada_classify([[0, 0]], classifier_array)

[[0.]]
[[-0.69314718]]
[[-1.66610226]]


array([[-1.]])

time: 5.35 ms


# Lets try AdaBoost on more complex dataset

## Load the dataset

In [66]:
data_path = "../data/adaboost/"
training_path = data_path + "horse_colic_training.txt"
testing_path = data_path + "horse_colic_testing.txt"

time: 769 µs


In [68]:
features_len = len(open(training_path).readline().split("\t"))
features_len

22

time: 3.43 ms


In [69]:
data_matrix, label_matrix = list(), list()

file_obj = open(training_path)
for line in file_obj.readlines():
    line_arr = list()
    cur_line = line.strip().split("\t")
    for feature_ind in range(features_len - 1):
        line_arr.append(float(cur_line[feature_ind]))
    data_matrix.append(line_arr)
    label_matrix.append(float(cur_line[-1]))

time: 10.4 ms


## Train using AdaBoost

In [73]:
classifier_array = adaboost(data_matrix, label_matrix, epochs=40)

min error: inf
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.43812709]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227]]
min error: [[0.40468227

min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [[0.28428094]]
min error: [

min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [[0.34969764]]
min error: [

min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [[0.36040207]]
min error: [

min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [[0.44672952]]
min error: [

...

min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [[0.46777342]]
min error: [

min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [[0.47348047]]
min error: [

min error: [[0.49813892]]
min error: [[0.49813892]]
min error: [[0.49813892]]
min error: [[0.49813892]]
min error: [[0.49813892]]
min error: [[0.4917269]]
min error: [[0.4917269]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0.46755929]]
min error: [[0

min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
min error: [[0.46333564]]
class_est:  [[ 1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
   1.  1.  1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1. -1.  1.  1.
   1.  1.  1. -1.  1. -1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1.
   1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1.  1. -1. -1.  1.  1.
  -1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1.  1.
  -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1. -1.  1.
   1.  1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1. -1.  1.  1. -1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1.
   1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1.  1. -1.  1.  1.  1.
   1.  1.  1.  1. -1.  1.  1.  1.  1.  1

min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [[0.46071977]]
min error: [

## Lets test it now!

In [74]:
data_matrix, label_matrix = list(), list()

file_obj = open(testing_path)
for line in file_obj.readlines():
    line_arr = list()
    cur_line = line.strip().split("\t")
    for feature_ind in range(features_len - 1):
        line_arr.append(float(cur_line[feature_ind]))
    data_matrix.append(line_arr)
    label_matrix.append(float(cur_line[-1]))

time: 7.91 ms


In [79]:
prediction = ada_classify(data_matrix, classifier_array)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [-0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.46166238]
 [ 0.4616623

## Calculate accuracy

In [78]:
error_mat = np.mat(np.ones((67, 1)))

time: 754 µs


In [83]:
error_mat[prediction != np.mat(label_matrix).T].sum() / 67

0.19402985074626866

time: 4.12 ms


So we have got 19 percent error rate, which translates to 81 percent accuracy by using only week learners!